In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fuel-data/train.csv
/kaggle/input/fuel-data/test.csv
/kaggle/input/fuel-data/sample_solution.csv


In [2]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor, VotingRegressor
from sklearn import linear_model 
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from xgboost import XGBRegressor
from scipy.stats import randint, uniform
from sklearn.linear_model import LinearRegression, Ridge, Lasso


In [3]:
sample_solution = pd.read_csv("/kaggle/input/fuel-data/sample_solution.csv", index_col='ID')

In [4]:
sample_solution

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
ID,,,,,,,,,,
1,-0.117370,0.348090,0.473673,0.079501,-0.411504,0.015352,0.454957,0.065651,-0.146684,-0.140500
2,-0.503910,-0.250186,-1.412727,-0.523577,-0.577571,-0.294264,-1.396187,-0.856044,-0.003241,-0.246948
3,1.484172,1.272972,1.188539,1.321349,1.472491,1.237584,1.192748,1.575889,0.773926,1.917254
4,0.841616,0.457436,0.534375,0.376650,1.593406,0.157950,0.516430,0.632370,0.376289,-0.446052
5,-0.024147,0.136198,1.174866,-0.197264,2.463520,0.418315,1.185596,0.509797,-0.434762,0.807128
...,...,...,...,...,...,...,...,...,...,...
496,0.022706,-0.047897,0.474691,-0.016551,-0.431127,-0.081999,0.462777,0.089287,-0.608682,0.516307
497,-1.475840,-1.294104,-1.230311,-1.231267,-0.303306,-0.977209,-1.217209,-1.553847,-1.126938,-1.407277
498,1.002035,0.870014,0.037003,0.963333,-0.261270,0.620495,0.029488,0.414339,0.726121,-0.088445


In [5]:
train_dataset = pd.read_csv('/kaggle/input/fuel-data/train.csv')
test_dataset  = pd.read_csv('/kaggle/input/fuel-data/test.csv',index_col='ID')

In [6]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.489143,0.607589,0.321670,-1.236055,1.601132,1.384662,0.305850,0.193460,0.580374,-0.762738
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-1.257481,-1.475283,-0.437385,-1.402911,0.147941,-1.143244,-0.439171,-1.379041,-1.280989,-0.503625
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,1.784349,0.450467,0.622687,1.375614,-0.428790,1.161616,0.601289,0.872950,0.660000,2.024576
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-0.066422,0.483730,-1.865442,-0.046295,-0.163820,-0.209693,-1.840566,0.300293,-0.351336,-1.551914
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.118913,-1.172398,0.301785,-1.787407,-0.493361,-0.528049,0.286344,-0.265192,0.430513,0.735073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,-0.028366,-0.327297,-0.316933,-1.294092,-0.530259,-0.421526,-0.320869,0.709627,-0.737244,-0.744289
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.449245,0.156778,-0.367445,-0.938615,-0.577451,-0.209996,-0.370505,-0.195531,-0.032834,0.269718
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,0.029135,0.164890,-0.092942,-1.134490,-0.437479,-0.695636,-0.101073,0.063650,0.624368,-0.477053
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,-0.232960,-0.464947,0.112536,-0.793522,-0.811272,-1.194914,0.100644,0.760116,-0.751394,-0.857598


In [7]:
train_dataset.describe()

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
count,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.18069,0.182910,0.179820,0.342090,0.114490,0.000245,-0.017319,0.001703,-0.004653,-0.018256,...,-0.016879,-0.002076,-0.014351,-0.006068,-0.015249,-0.003497,-0.013568,-0.017236,-0.001507,-0.001795
std,0.16320,0.163704,0.166283,0.141119,0.080219,0.999423,1.006409,0.998859,1.006902,1.009294,...,0.993787,1.004512,0.999360,1.009176,0.986480,1.009126,1.000613,0.998759,1.001096,0.990433
min,0.00000,0.000000,0.000000,0.010000,0.000000,-2.943737,-1.718895,-3.008683,-3.029468,-3.572440,...,-2.550897,-3.079759,-3.041624,-2.835701,-1.730111,-2.808210,-2.994571,-3.621080,-3.292727,-2.476429
25%,0.03000,0.040000,0.020000,0.220000,0.050000,-0.694658,-0.765154,-0.701948,-0.693361,-0.713149,...,-0.766128,-0.735109,-0.624235,-0.783547,-0.683165,-0.697379,-0.622453,-0.725564,-0.702384,-0.733653
50%,0.14000,0.150000,0.140000,0.350000,0.120000,0.011977,-0.030235,0.021335,0.016774,0.194936,...,-0.021089,0.001684,0.146135,-0.028158,-0.250650,-0.011649,0.133470,-0.001548,-0.002604,-0.010459
75%,0.29000,0.300000,0.290000,0.500000,0.180000,0.685717,0.653960,0.673125,0.659227,1.032029,...,0.714763,0.723807,0.727597,0.664659,0.358701,0.695182,0.704130,0.684894,0.706084,0.693839
max,0.50000,0.500000,0.500000,0.500000,0.290000,2.981146,3.051090,2.868901,2.982258,1.032029,...,2.856588,2.769156,1.638646,3.769643,3.600439,3.433292,3.293228,3.340657,3.276199,2.708703


In [8]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 65 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    2000 non-null   float64
 1   Component2_fraction    2000 non-null   float64
 2   Component3_fraction    2000 non-null   float64
 3   Component4_fraction    2000 non-null   float64
 4   Component5_fraction    2000 non-null   float64
 5   Component1_Property1   2000 non-null   float64
 6   Component2_Property1   2000 non-null   float64
 7   Component3_Property1   2000 non-null   float64
 8   Component4_Property1   2000 non-null   float64
 9   Component5_Property1   2000 non-null   float64
 10  Component1_Property2   2000 non-null   float64
 11  Component2_Property2   2000 non-null   float64
 12  Component3_Property2   2000 non-null   float64
 13  Component4_Property2   2000 non-null   float64
 14  Component5_Property2   2000 non-null   float64
 15  Comp

In [9]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component1_Property9,Component2_Property9,Component3_Property9,Component4_Property9,Component5_Property9,Component1_Property10,Component2_Property10,Component3_Property10,Component4_Property10,Component5_Property10
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,-0.265376,0.123432,0.028533,-0.173365,1.297923,0.323299,-0.315146,0.625518,-0.514342,-0.777057
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,-0.787677,-0.757905,-0.280561,-1.965970,0.543475,-0.906851,0.962341,-0.183757,0.310871,-1.329042
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,-0.710026,-1.422693,0.874071,-1.016144,0.093525,1.048525,-1.321851,0.356640,-0.869543,-0.177255
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,-0.551366,0.257105,-0.077337,-0.721031,-0.760365,-0.507690,1.346556,-0.001529,-1.008445,1.726105
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,-1.811468,-0.181223,-0.475933,0.234775,-0.909020,1.238203,-1.805664,0.980417,-1.354932,-0.657513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,-2.690249,-0.231333,-1.365939,-0.853755,-1.344675,-1.947526,-1.074584,-0.421069,-0.603527,0.838356
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,-0.458610,-1.422693,1.952542,-0.283461,0.367323,2.689269,1.698609,-0.328886,-0.879281,-1.658223
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,-0.437798,-0.810355,-0.926471,-1.480680,-1.613630,1.016878,0.989316,0.408454,-0.925924,-0.022020


In [10]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 500
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    500 non-null    float64
 1   Component2_fraction    500 non-null    float64
 2   Component3_fraction    500 non-null    float64
 3   Component4_fraction    500 non-null    float64
 4   Component5_fraction    500 non-null    float64
 5   Component1_Property1   500 non-null    float64
 6   Component2_Property1   500 non-null    float64
 7   Component3_Property1   500 non-null    float64
 8   Component4_Property1   500 non-null    float64
 9   Component5_Property1   500 non-null    float64
 10  Component1_Property2   500 non-null    float64
 11  Component2_Property2   500 non-null    float64
 12  Component3_Property2   500 non-null    float64
 13  Component4_Property2   500 non-null    float64
 14  Component5_Property2   500 non-null    float64
 15  Component1_

# polynomial _Features creation

In [11]:
# poly = PolynomialFeatures(degree=(2,4))
# poly.fit(train_dataset)

In [12]:
# train_dataset = poly.transform(train_dataset)
# scaler = preprocessing.StandardScaler().fit(train_dataset)
# train_dataset = scaler.transform(train_dataset)

In [13]:
# test_dataset=test_dataset.interpolate(method='linear')#.isna().sum()
# test_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# test_dataset=test_dataset.interpolate(method='linear')#.isna().sum()
# test_dataset = poly.transform(test_dataset)
# test_dataset = scaler.transform(test_dataset)

## Splitting the  test data and make testset and validationset

In [14]:
train_X, val_X, train_y, val_y = train_test_split(train_dataset.iloc[:,:55],train_dataset.iloc[:,55:], random_state=42, test_size=0.05,)

In [15]:
# sns.pairplot(train_dataset)


# Use Tablenet The transformet approach

In [ ]:
xgbr = XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    random_state=42,
    device='cuda',
)

multi_output_model_xgbr = MultiOutputRegressor(estimator=xgbr)
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

param_distributions = {
    'estimator__n_estimators': randint(30, 200),
    'estimator__max_depth': randint(3, 20),
    'estimator__learning_rate': uniform(0.0001, 0.3),
    'estimator__reg_alpha': uniform(0.1, 0.5),
    'estimator__reg_lambda': uniform(0.1, 0.5),
    'estimator__max_leaves': randint(3, 20),
    'estimator__max_bin': randint(3, 100),
    'estimator__colsample_bytree': uniform(0.5, 0.5) 
}

searcher = RandomizedSearchCV(
    multi_output_model_xgbr,
    param_distributions,
    n_iter=30,
    scoring=scorer,
    n_jobs=-1,
    refit=True,
    cv=3,
    verbose=2,
    random_state=42,
)

search = searcher.fit(train_X,train_y)
print(search.best_params_)
best_model_xgbr = searcher.best_estimator_
history = best_model_xgbr.fit(train_X,train_y)
xgb_pred = best_model_xgbr.predict(val_X)
# mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
print("MSE: ", mean_squared_error(val_y, xgb_pred))
print("MAE: ", mean_absolute_error(val_y, xgb_pred))
print("R2 : ", r2_score(val_y, xgb_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, xgb_pred))

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [15:55:05] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [15:55:05] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for boo

In [ ]:
linr = LinearRegression(
    n_jobs=-1
)
multi_output_lr = MultiOutputRegressor(estimator=linr)
param_distributions = {
    'estimator__fit_intercept': [True, False],
    'estimator__copy_X': [True, False],
    'estimator__normalize': [True, False],
}
searcher_lr = RandomizedSearchCV(
    estimator=multi_output_lr,
    param_distributions=param_distributions,
    n_iter=10,          # fewer total combinations, so fewer iters
    scoring=scorer,
    cv=3,
    n_jobs=-1,
    verbose=2,
    refit=True,
    random_state=42
)
searcher_lr.fit(train_X, train_y)
print(searcher_lr.best_params_)
best_lr = searcher_lr.best_estimator_
lr_pred = best_lr.predict(val_X)
print("MSE: ", mean_squared_error(val_y, lr_pred))
print("MAE: ", mean_absolute_error(val_y, lr_pred))
print("R2 : ", r2_score(val_y, lr_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, lr_pred))

In [ ]:
pipeline_rid = make_pipeline(
    SimpleImputer(strategy='mean'),
    Ridge(
        alpha=0.3,
        max_iter=2000,
        solver='auto',
        random_state=42
    )
)

param_distributions_rid = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'ridge__alpha': uniform(0.01, 10),    
    'ridge__solver': ['auto', 'svd', 'cholesky', 'lsqr', 
                      'sparse_cg', 'sag', 'saga', 'lbfgs']
}

searcher_rid = RandomizedSearchCV(
    estimator=pipeline_rid,
    param_distributions=param_distributions_rid,
    n_iter=30,               
    scoring='neg_mean_absolute_percentage_error',  
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42,
    refit=True
)


search_rid = searcher_rid.fit(train_X, train_y)
print("Best params:", search_rid.best_params_)
best_ridge_pipeline = search_rid.best_estimator_

rid_pred = best_ridge_pipeline.predict(val_X)

print("MSE:  ", mean_squared_error(val_y, rid_pred))
print("MAE:  ", mean_absolute_error(val_y, rid_pred))
print("R2:   ", r2_score(val_y, rid_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rid_pred))


In [ ]:
pipeline_lasso = make_pipeline(
    SimpleImputer(),  # default strategy='mean'
    Lasso(max_iter=5000, random_state=42)
)

param_distributions_lasso = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'lasso__alpha': uniform(0.0001, 1.0),       # α between 0.0001 and 1.0001
    'lasso__selection': ['cyclic', 'random'],   # coordinate descent update strategy
    'lasso__warm_start': [True, False],         # reuse solution of previous call
    'lasso__tol': uniform(1e-5, 1e-2)           # tolerance for optimization
}

searcher_lasso = RandomizedSearchCV(
    estimator=pipeline_lasso,
    param_distributions=param_distributions_lasso,
    n_iter=30,                                    # number of sampled configs
    scoring='neg_mean_absolute_percentage_error', # minimize MAPE
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42,
    refit=True
)

search_lasso = searcher_lasso.fit(train_X, train_y)
print("Best Lasso params:", search_lasso.best_params_)


best_lasso = search_lasso.best_estimator_
lasso_pred = best_lasso.predict(val_X)
# 8) Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, lasso_pred))
print("MAE:  ", mean_absolute_error(val_y, lasso_pred))
print("R2:   ", r2_score(val_y, lasso_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, lasso_pred))

In [ ]:
pipeline_rf = make_pipeline(
    SimpleImputer(strategy='median'),
    RandomForestRegressor(
        n_estimators=100,
        criterion='squared_error',
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features=1.0,
        bootstrap=True,
        n_jobs=-1,
        random_state=42,
        verbose=0,
        warm_start=False
    )
)

param_distributions_rf = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'randomforestregressor__criterion': ["squared_error", "absolute_error", "friedman_mse", "poisson"],
    'randomforestregressor__n_estimators': randint(30, 500),
    'randomforestregressor__min_samples_split': randint(2, 200),
    'randomforestregressor__min_samples_leaf': randint(1, 50),
    'randomforestregressor__max_features': ["sqrt", "log2", None],
    'randomforestregressor__max_leaf_nodes': randint(30, 1000),
    'randomforestregressor__max_depth': randint(3, 500),
}

searcher_rf = RandomizedSearchCV(
    estimator=pipeline_rf,
    param_distributions=param_distributions_rf,
    n_iter=50,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=5,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    refit=True
)

search_rf = searcher_rf.fit(train_X, train_y)

print("Best RF params:", search_rf.best_params_)

best_rf = search_rf.best_estimator_

rf_pred = best_rf.predict(val_X)

# 8) Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, rf_pred))
print("MAE:  ", mean_absolute_error(val_y, rf_pred))
print("R²:   ", r2_score(val_y, rf_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rf_pred))


In [ ]:
pipeline_etc = make_pipeline(
    SimpleImputer(strategy='mean'),
    ExtraTreesRegressor(
        n_estimators=100,
        criterion='squared_error',
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features=1.0,
        bootstrap=False,
        random_state=42,
        n_jobs=-1,
        warm_start=False
    )
)

param_distributions_etc = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'extratreesregressor__n_estimators': randint(50, 500),
    'extratreesregressor__max_depth': [None] + list(range(5, 31, 5)),
    'extratreesregressor__min_samples_split': randint(2, 20),
    'extratreesregressor__min_samples_leaf': randint(1, 20),
    'extratreesregressor__max_features': ['sqrt', 'log2', None] + list(uniform(0.1, 0.9).rvs(5)),
    'extratreesregressor__bootstrap': [False, True],
    'extratreesregressor__ccp_alpha': uniform(0.0, 0.1),  # cost-complexity pruning
}

searcher_etc = RandomizedSearchCV(
    estimator=pipeline_etc,
    param_distributions=param_distributions_etc,
    n_iter=50,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=5,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    refit=True
)

search_etc = searcher_etc.fit(train_X, train_y)

print("Best ExtraTrees params:", search_etc.best_params_)

best_etc = search_etc.best_estimator_

etc_pred = best_etc.predict(val_X)

# 8) Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, etc_pred))
print("MAE:  ", mean_absolute_error(val_y, etc_pred))
print("R²:   ", r2_score(val_y, etc_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, etc_pred))


In [ ]:
pipeline_svr = make_pipeline(
    SimpleImputer(strategy='mean'),
    SVR(
        kernel='rbf',
        degree=3,
        gamma='scale',
        coef0=0.0,
        tol=0.001,
        C=1.0,
        epsilon=0.1,
        shrinking=True,
        cache_size=200,
        verbose=False,
        max_iter=-1,
    )
)

multioutput_svr = MultiOutputRegressor(pipeline_svr)

param_distributions_svr = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'svr__C': uniform(0.01, 100),                   
    'svr__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'svr__degree': randint(2, 6),                   
    'svr__gamma': ['scale', 'auto'] + list(uniform(0.0001, 1).rvs(5)),
    'svr__coef0': uniform(0.0, 1.0),                
    'svr__shrinking': [True, False],
    'svr__tol': uniform(1e-5, 1e-2),                
    'svr__max_iter': [-1, 1000, 2000],              
    'svr__epsilon': uniform(0.0, 1.0),
}

searcher_svr = RandomizedSearchCV(
    estimator=multioutput_svr,
    param_distributions=param_distributions_svr,
    n_iter=20,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=5,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    refit=True
)

search_svr = searcher_svr.fit(train_X, train_y)

print("Best SVR params:", search_svr.best_params_)

best_svr = search_svr.best_estimator_

svr_pred = best_svr.predict(val_X)

# 9) Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, svr_pred))
print("MAE:  ", mean_absolute_error(val_y, svr_pred))
print("R²:   ", r2_score(val_y, svr_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, svr_pred))


In [ ]:
from sklearn.ensemble import VotingRegressor

estimators = [
    ('xgb', best_xgb),
    ('rf',  best_rf),
    ('etc', best_etc),
    ('lr',  best_lr),
    ('svc', best_svr)
]

voting = VotingRegressor(estimators=estimators, n_jobs=-1)
voting.fit(train_X, train_y)

# And then evaluate
voting_pred = voting.predict(val_X)
print("Voting MSE:  ", mean_squared_error(val_y, voting_pred))
print("Voting MAE:  ", mean_absolute_error(val_y, voting_pred))
print("Voting R²:   ", r2_score(val_y, voting_pred))
print("Voting MAPE: ", mean_absolute_percentage_error(val_y, voting_pred))


# Applying deeplearning

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential, regularizers, callbacks

# 1) Encoder as its own Sequential
encoder_seq = Sequential([
    layers.Input(shape=(55,), name="enc_input"),
    layers.Dense(32, activation="relu", name="enc_dense_32"),
    layers.Dense(16, activation="relu", name="enc_dense_16"),
    layers.Dense(8,  activation="relu", name="bottleneck")
], name="encoder_seq")

# 2) Decoder as its own Sequential
decoder_seq = Sequential([
    layers.Input(shape=(8,), name="dec_input"),
    layers.Dense(16, activation="relu", name="dec_dense_16"),
    layers.Dense(32, activation="relu", name="dec_dense_32"),
    layers.Dense(55, activation="linear", name="dec_output")
], name="decoder_seq")

# 3) Full autoencoder by chaining encoder→decoder
autoencoder_seq = Sequential([encoder_seq, decoder_seq], name="autoencoder_seq")
autoencoder_seq.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="mse"
)

# 4) Train autoencoder
history_ae_train = autoencoder_seq.fit(
    train_X, train_X,
    validation_data=(val_X, val_X),
    epochs=100,
    batch_size=32,
    callbacks=[
        callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    ]
)



In [ ]:
history_ae_test = autoencoder_seq.fit(
    test_dataset, test_dataset,
    validation_data=(val_X, val_X),
    epochs=100,
    batch_size=32,
    callbacks=[
        callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    ]
)


In [ ]:
# 5) Freeze the encoder weights
encoder_seq.trainable = False

# 6) Build the regression head on top of the frozen encoder
regressor = Sequential(name="regressor_seq")
regressor.add(encoder_seq)  # now frozen
regressor.add(layers.Dense(16, activation="relu", name="reg_dense_16"))
regressor.add(layers.Dense(10, activation="linear", name="reg_output"))  # 10 float outputs

# 7) Compile the regression model
regressor.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="mse",
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")]
)

# 8) Train on your multi‐output targets
history_reg = regressor.fit(
    train_data, train_targets_10,
    validation_data=(val_data, val_targets_10),
    epochs=100,
    batch_size=32,
    callbacks=[
        callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    ]
)

# 9) Now you can predict 10 float outputs:
predictions = regressor.predict(new_data)  # shape (n_samples, 10)


In [ ]:
voting_clf.fit(train_X,train_y)

In [ ]:
index = pd.DataFrame(test_dataset).index

In [ ]:
index_col_test_dataset

In [ ]:
voting_clf_pred = voting_clf.predict(test_dataset)

In [ ]:
voting_clf_pred = pd.Series(voting_clf_pred).map(rev_personality)

In [ ]:
voting_clf_pred

In [ ]:
pd.DataFrame(voting_clf_pred)

In [ ]:
df = pd.DataFrame({'Personality': voting_clf_pred.values}, index=index_col_test_dataset)
df.index.name = 'id'  # Optional: set index name if required
df.to_csv('/kaggle/working/output.csv', index=True)
df

In [ ]:
print(accuracy_score(val_y,voting_clf_pred))
print(precision_score(val_y,voting_clf_pred))
print(recall_score(val_y,voting_clf_pred))
print(f1_score(val_y,voting_clf_pred))

In [ ]:
st_clf = StackingRegressor(estimators=estimators, final_estimator=LogisticRegression(),n_jobs=-1)

In [ ]:
# st_clf.fit(train_X,train_y)

In [ ]:
st_clf_pred = voting_clf.predict(val_X)

In [ ]:
print(accuracy_score(val_y,voting_clf_pred))
print(precision_score(val_y,voting_clf_pred))
print(recall_score(val_y,voting_clf_pred))
print(f1_score(val_y,voting_clf_pred))